In [1]:
from primeqa.components.retriever.dense import ColBERTRetriever
from primeqa.components.retriever.dense import DPRRetriever
from primeqa.components.reader.prompt import PromptGPTReader
from primeqa.pipelines.qa_pipeline import QAPipeline
import json

{"time":"2023-02-13 12:31:56,103", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-02-13 12:31:56,193", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


In [2]:
# setup ColBERT index
index_root = "/dccstor/colbert-ir/franzm/indexes/oct2_10_11/oct2_10_11_exp/indexes/"
index_name = "oct2_10_11_indname"
collection = "/dccstor/avi7/neural_ir/colbert/data/psgs_w100.tsv"


retriever = ColBERTRetriever(index_root = index_root, 
                                     index_name = index_name, 
                                     collection = collection, 
                                     max_num_documents = 3)
retriever.load()

[Feb 13, 12:32:03] #> base_config.py from_path /dccstor/colbert-ir/franzm/indexes/oct2_10_11/oct2_10_11_exp/indexes//oct2_10_11_indname/metadata.json
[Feb 13, 12:32:03] #> base_config.py from_path args loaded! 
[Feb 13, 12:32:03] #> base_config.py from_path args replaced ! 
[Feb 13, 12:32:15] #>>>>> at ColBERT name (model type) : /dccstor/colbert-ir/franzm/experiments/oct2_7_12_1.5e-06/none/2022-10/09/15.21.39/checkpoints/colbert.dnn.batch_91287.model
[Feb 13, 12:32:15] #>>>>> at BaseColBERT name (model type) : /dccstor/colbert-ir/franzm/experiments/oct2_7_12_1.5e-06/none/2022-10/09/15.21.39/checkpoints/colbert.dnn.batch_91287.model
[Feb 13, 12:32:24] factory model type: xlm-roberta-large
[Feb 13, 12:32:39] get query model type: xlm-roberta-large
[Feb 13, 12:32:40] get doc model type: xlm-roberta-large
[Feb 13, 12:32:53] #> Loading codec...
[Feb 13, 12:32:53] #> base_config.py from_path /dccstor/colbert-ir/franzm/indexes/oct2_10_11/oct2_10_11_exp/indexes//oct2_10_11_indname/metadata.js

In [2]:
#checkpoint = "/dccstor/colbert-ir/franzm/experiments/jan3_7_10/qry_encoder"
checkpoint = "/dccstor/colbert-ir/franzm/experiments/oct2_18_10/qry_encoder"
index_root = "/dccstor/colbert-ir/franzm/experiments/"
index_name = "oct2_18_20"
collection = "/dccstor/avi7/neural_ir/colbert/data/psgs_w100.tsv"

retriever = DPRRetriever(index_root = index_root, 
                                     index_name = index_name,  
                                     checkpoint = checkpoint,
                                     collection = collection,
                                     max_num_documents = 3)

retriever.load()

{"time":"2023-02-13 12:22:49,144", "name": "primeqa.ir.dense.dpr_top.torch_util.hypers_base", "level": "INFO", "message": "world_rank 0 cuda_is_available True cuda_device_cnt 1 on cccxc564, CUDA_VISIBLE_DEVICES = 0"}
{"time":"2023-02-13 12:23:48,217", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Using sharded faiss, reading shards from /dccstor/colbert-ir/franzm/experiments/oct2_18_20"}
{"time":"2023-02-13 12:23:48,218", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_1_of_40.json.gz.records"}
{"time":"2023-02-13 12:23:51,505", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_15_of_40.json.gz.records"}
{"time":"2023-02-13 12:23:54,465", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_4_of_40.json.gz.records"}
{"time":"2023-02-13 12:23:56,992", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "messag

We provide three examples of loading models to generate answers from the prompt: GPT, Flan-T5-xxl from HuggingFace, and Flan-T5-xxl from BAM services

In [3]:
# setup a Prompt Reader: here we use a GPT reader
reader = PromptGPTReader(api_key='API KEY HERE')
reader.load()

In [4]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

21015325it [01:04, 325853.20it/s]


In [5]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAnswer: Yes, smoking marijuana can impair driving. Studies have shown that marijuana use can lead to reduced attention, coordination, and reaction time, as well as altered perception and judgment, which can all increase the risk of a car accident."
    }
]


In [7]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 57.50it/s]


[
    {
        "example_id": 0,
        "text": "\n\nYes, smoking marijuana can impair driving. Studies have found that drivers who tested positive for marijuana were no more likely to crash than those who had not used any drugs or alcohol prior to driving. However, law enforcement officers base arrests on observed impairment, and it is illegal to be driving if consumption of marijuana is impairing your ability to drive.",
        "passages": [
            "\"Driving under the influence\"\n \"driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Prescription medications such as opioids and benzodiazepines often cause side effects such as excessive drowsiness, and, in the case of opioids, nausea. Other prescripti

In [8]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAnswer: At high altitudes, jets leave a trail of water vapor, ice crystals, and other particles behind them. These trails are known as contrails and form when hot, humid air from the jet exhaust mixes with the cold, dry air in the upper atmosphere."
    }
]


In [9]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 29.80it/s]


[
    {
        "example_id": 0,
        "text": "\n\nContrails are caused by the combustion of fuel in jet engines at high altitude. They are visible cores of wingtip vortices that trail behind the wingtips and wing flaps rather than behind the engines.",
        "passages": [
            "\"Mitigation of aviation's environmental impact\"\n \"of fuel and are produced in greater quantities in engines with low pressure gains in the compressors and/or relatively low temperatures in the combustor. As with particulates, UHC has all but been eliminated in modern jet engines through improved design and technology. Aircraft flying at high altitude form condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although the extent of the damage to the environment is as yet unknown. Contrails can also trigger\"",
            "

In [10]:
# start asking questions
questions = ["number of participating countries in tour de france 2017 ?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAnswer: 22."
    }
]


In [34]:
# start asking questions
questions = ["number of participating countries in tour de france 2017 ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 35.16it/s]


[
    {
        "example_id": 0,
        "text": "\n\nAnswer: 32 countries",
        "passages": [
            "\"2017 Tour de France\"\n \"de France. The total number of riders that finished the race was 167. The riders came from 32 countries. Six countries had more than 10 riders in the race: France (39), Italy (18), Belgium (16), Germany (16), the Netherlands (15), and Spain (13). The average age of riders in the race was 29.4 years, ranging from the 22-year-old \u00c9lie Gesbert () to the 40-year-old Haimar Zubeldia (). had the youngest average age while had the oldest. The teams entering the race were: In the lead up to the 2017 Tour de France, Chris Froome () was seen by many pundits\"",
            "\"2018 Tour de France\"\n \"reduced the number of riders per team for Grand Tours from 9 to 8, resulting in a start list total of 176, instead of the usual 198. Of these, 35 competed in their first Tour de France. The total number of riders that finished the race was 145. The riders 

In [64]:
# start asking questions
questions = ["how many states in the us have the death penalty 2017 ?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAs of 2017, 31 states in the US have the death penalty."
    }
]


In [16]:
# start asking questions
questions = ["how many states in the us have the death penalty 2018 ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 94.43it/s]


[
    {
        "example_id": 0,
        "text": "\n\n31 states",
        "passages": [
            "\"Capital punishment debate in the United States\"\n \"Capital punishment debate in the United States Capital punishment debate in the United States existed as early as the colonial period. As of 2017 it remains a legal penalty in 31 states, the federal government, and military criminal justice systems. Gallup, Inc. monitors support for the death penalty in the United States since 1937 by asking \"\"Are you in favor of the death penalty for a person convicted of murder?\"\" Opposition to the death penalty peaked in 1966, with 47% of Americans opposing it; by comparison, 42% supported the death penalty and 11% had \"\"no opinion.\"\" The death penalty increased\"",
            "\"United States\"\n \"3,246 violent and property crimes per 100,000 residents in 2012, for a total of over 9 million total crimes. Capital punishment is sanctioned in the United States for certain federal and mili

In [65]:
# start asking questions
questions = ["How many area codes are in New Hampshire ?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nNew Hampshire has two area codes: 603 and 380."
    }
]


In [66]:
questions = ["How many area codes are in New Hampshire ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.50it/s]


[
    {
        "example_id": 0,
        "text": "\n\nAnswer: One",
        "passages": [
            "\"Area code 603\"\n \"previously allocated telephone numbers, as well as number pooling, the exhaustion time frame has been moved beyond 2025. Since New Hampshire has only one area code, callers in the state can reach any other telephone within the 603 area code with 7-digit dialing. Area code 603 Area code 603 is the sole area code for the U.S. state of New Hampshire in the North American Numbering Plan (NANP). It was created as one of the original 86 numbering plan areas in October 1947. As of April 2011, the 603 code was nearing exhaustion and a second area code for New\"",
            "\"Area code 603\"\n \"Area code 603 Area code 603 is the sole area code for the U.S. state of New Hampshire in the North American Numbering Plan (NANP). It was created as one of the original 86 numbering plan areas in October 1947. As of April 2011, the 603 code was nearing exhaustion and a second a

In [60]:
# start asking questions
questions = ["what engine is in the audi r8 v10"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAnswer: The Audi R8 V10 is equipped with a 5.2-liter naturally aspirated V10 engine that produces 540 horsepower and 398 lb-ft of torque."
    }
]


In [61]:
questions = ["what engine is in the audi r8 v10"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 108.82it/s]


[
    {
        "example_id": 0,
        "text": "\n\nThe engine in the Audi R8 V10 is a 5.2 litre FSI engine.",
        "passages": [
            "\"Audi R18\"\n \"gearbox is electrically controlled instead of pneumatically controlled, saving weight by eliminating the pneumatic system. Despite the capacity reduction, the 3.7L V6 is claimed to develop more than of power. This is less than the outgoing R15, but the V6 engine's fuel consumption will more than likely be lower than that of the outgoing V10 engine on the R15. The new engine has a single Garrett (Honeywell Turbo Technologies) TR30R VGT turbocharger, as opposed to the twin TR30R configuration of both the Peugeot 908 HDi FAP and the previous Audi R15 TDI. The R18's V6 engine exhausts inwards between the\"",
            "\"Audi RS 6\"\n \"steering wheel movements. The facelifted Audi A6, released as a 2009 model, received similar front and rear-end LED lighting styling to that pioneered on the RS 6. Luggage capacity, measured a

In [62]:
# start asking questions
questions = ["who was the last person hanged in england"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": " for treason\n\nAnswer: William Joyce, also known as Lord Haw-Haw, was the last person to be hanged in England for treason in 1946."
    }
]


In [63]:
questions = ["who was the last person hanged in england"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.51it/s]


[
    {
        "example_id": 0,
        "text": "\n\nThe last person hanged in England was John Robson Walby (alias Gwynne Owen Evans).",
        "passages": [
            "\"HM Prison Manchester\"\n \"that of twenty-year-old murderer Michael Johnson, who was hanged by William Calcraft on 29 March 1869. Twenty-nine hangings took place over the next twenty years and 71 took place in the 20th century, bringing the total number to 100. During the second half of the century, the number of executions decreased, with no hangings between 1954 and 28 November 1962, when James Smith was executed. John Robson Walby (alias Gwynne Owen Evans), one of the last two people to be hanged in England, was executed here on 13 August 1964. Out of the 100 hangings, four were double hangings, while\"",
            "\"Harry Allen (executioner)\"\n \"Cyprus. Pierrepoint was no longer available because he had resigned in February 1956. Allen's most controversial hanging came in April 1962, when James Hanratty 

In [26]:
#setup a FlanT5PromptReader: note this loads a Flan T5 xxl model
from primeqa.components.reader.prompt import PromptFLANT5Reader
reader = PromptFLANT5Reader()
reader.load()

KeyboardInterrupt: 

In [ ]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

In [17]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

[Feb 05, 15:18:28] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Feb 05, 15:18:28] #> Input: $ Does smoking marijuana impair driving ?, 		 True, 		 None
[Feb 05, 15:18:28] #> Output IDs: torch.Size([32]), tensor([     0,   9748, 101790, 200166, 227071,    566, 109637, 132442,    705,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[Feb 05, 15:18:28] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Feb 05, 15:18:29] #>>>> colbert query ==
[Feb 05, 15:18:29] #>>>>> input_ids: torch.Size([32]), tensor([     0,   9748, 101790, 200166, 227071,    566, 109637, 132442,    705,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,  

100%|██████████| 1/1 [00:00<00:00, 66.32it/s]
/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[
    {
        "example_id": 0,
        "text": [
            "\"[a]ny amount of marijuana consumption puts you at risk of driving impaired.\"\""
        ],
        "passages": [
            "\"In the US state of Colorado, the state government indicates that \"\"[a]ny amount of marijuana consumption puts you at risk of driving impaired.\"\" Colorado law states that \"\"drivers with five nanograms of active tetrahydrocannabinol (THC) in their whole blood can be prosecuted for driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Section: Other drugs.:Prescription medications.\"",
            "\"Section: Legal standards.:Americas.:United States.:State.:Georgia. Georgia Statute \u00a7 40-6-391 makes it to drive whi

In [18]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude? ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

100%|██████████| 1/1 [00:00<00:00, 77.71it/s]


[
    {
        "example_id": 0,
        "text": [
            "condensation trails"
        ],
        "passages": [
            "\"Aircraft flying at high altitude form condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although the extent of the damage to the environment is as yet unknown. Contrails can also trigger the formation of high-altitude Cirrus cloud thus creating a greater climatic effect. A 2015 study found that artificial cloudiness caused by contrail \"\"outbreaks\"\" reduce the difference between daytime and nighttime temperatures. The former are decreased and the latter are increased, in comparison\"",
            "\"Contrails, or condensation trails, are \"\"streaks of condensed water vapor created in the air by an airplane or rocket at high altitudes.\"\" Fossil fuel combustion (as in pisto

In [19]:
#setup a FlanT5PromptReader using the bam service (https://bam.res.ibm.com/): note this loads a Flan T5 xxl model
from primeqa.components.reader.prompt import PromptFLANT5Reader
reader = PromptFLANT5Reader(use_bam=True, api_key='API KEY HERE')
reader.load()

In [20]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

14635it [00:00, 283158.38it/s]


In [21]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

100%|██████████| 1/1 [00:00<00:00, 36.33it/s]
/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bam-api.res.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[
    {
        "example_id": 0,
        "text": "\"[a]ny amount of marijuana consumption puts you at risk of driving impaired.\"\" Colorado law states that \"\"drivers with five nanograms of active tetrahydrocannabinol (THC) in their whole blood can be prosecuted for driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Section: Other drugs.:Prescription medications.\", \"Section: Legal standards.:Americas.:United States.:State.:Georgia. Georgia Statute  40-6-391 makes it to drive while there is \"\"\"\"any amount\"\" of marijuana . . . present in the person's blood or urine, or both, \"\"including the metabolites and derivatives of each or both\"\"\"\" Subsection (b) of the statute provides that being legally e

In [22]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude? ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

100%|██████████| 1/1 [00:00<00:00, 58.55it/s]
/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bam-api.res.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[
    {
        "example_id": 0,
        "text": "condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although the extent of the damage to the environment is as yet unknown. Contrails can also trigger the formation of high-altitude Cirrus cloud thus creating a greater climatic effect. A 2015 study found that artificial cloudiness caused by contrail \"\"outbreaks\"\" reduce the difference between daytime and nighttime temperatures. The former are decreased and the latter are increased, in comparison\"",
        "passages": [
            "\"Aircraft flying at high altitude form condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although 